In [ ]:
import cv2
import pickle

# You can change the dimensions of the selection box for saving here
width, height = 90, 110

# Make a file to store the positions of a person to save 
try:
    with open('DetectPos', 'rb') as f:
        posList = pickle.load(f)
except:
    posList = []

def mouseClick(events, x, y, flags, params):
    if events == cv2.EVENT_LBUTTONDOWN:
        posList.append((x, y))
    if events == cv2.EVENT_RBUTTONDOWN:
        for i, pos in enumerate(posList):
            x1, y1 = pos
            if x1 < x < x1 + width and y1 < y < y1 + height:
                posList.pop(i)

    # Writing the selected space in the file for further use
    with open('DetectPos', 'wb') as f:
        pickle.dump(posList, f)


while True:
    # Screenshot taken from the video
    img = cv2.imread('dataset/image.jpg')
    for pos in posList:
        cv2.rectangle(img, pos, (pos[0] + width, pos[1] + height), (255, 0, 255), 2)

    cv2.imshow("Image", img)
    cv2.setMouseCallback("Image", mouseClick)

    # Check for the 'q' key press to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()


In [ ]:
import cv2
import pickle
import numpy as np

def nothing(x):
    pass

width, height = 90, 110

# Make a file to store the positions
try:
    with open('DetectPos', 'rb') as f:
        posList = pickle.load(f)
except:
    posList = []

# Load the input image
image_path = 'dataset/image.jpg'  # Replace with the actual path to your image
image = cv2.imread(image_path)

# Load the background image
background_path = 'dataset/background2.jpg'  # Replace with the actual path to your background image
background = cv2.imread(background_path)

# Resize the background image to match the size of the input image
background = cv2.resize(background, (image.shape[1], image.shape[0]))

# Create a copy of the input image
output_image = image.copy()

# Create a named window
cv2.namedWindow('Virtual Background')

# Create trackbars
cv2.createTrackbar('LH', 'Virtual Background', 0, 255, nothing)
cv2.createTrackbar('LS', 'Virtual Background', 0, 255, nothing)
cv2.createTrackbar('LV', 'Virtual Background', 0, 255, nothing)
cv2.createTrackbar('UH', 'Virtual Background', 255, 255, nothing)
cv2.createTrackbar('US', 'Virtual Background', 255, 255, nothing)
cv2.createTrackbar('UV', 'Virtual Background', 255, 255, nothing)

while True:
    # Get the current trackbar positions
    l_h = cv2.getTrackbarPos('LH', 'Virtual Background')
    l_s = cv2.getTrackbarPos('LS', 'Virtual Background')
    l_v = cv2.getTrackbarPos('LV', 'Virtual Background')
    u_h = cv2.getTrackbarPos('UH', 'Virtual Background')
    u_s = cv2.getTrackbarPos('US', 'Virtual Background')
    u_v = cv2.getTrackbarPos('UV', 'Virtual Background')

    # Convert the input image to HSV
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Create a mask based on the current trackbar values
    mask = cv2.inRange(hsv, (l_h, l_s, l_v), (u_h, u_s, u_v))

    # Exclude regions from DetectPos from the mask
    for pos in posList:
        x1, y1 = pos
        x2, y2 = x1 + width, y1 + height
        mask[y1:y2, x1:x2] = 255

    # Perform person detection by applying the mask to the output image
    output_image = image.copy()
    output_image[mask == 0] = background[mask == 0]

    # Display the modified output image
    cv2.imshow('Virtual Background', output_image)

    # Check for the 'q' key press to exit the loop
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyAllWindows()
